In [1]:
from main import *
from useful_functions import *
from visualization import *

import bct
import hdbscan
from sklearn.manifold import TSNE

%matplotlib inline
plt.rcParams['font.size'] = 6

# Setting paths

In [2]:
top_directory = '/media/anleg84/Backup Plus/CERVO/Data/Networks2024/'
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

In [3]:
atlas = MapzebrainAtlas(atlas_path)
centroids = np.concatenate([atlas.regionCentroids['left'], atlas.regionCentroids['right']], axis=0)
centroids[:, 1] = 974 - centroids[:, 1]

In [4]:
datasets = get_datasets(top_directory, keywords=['920nm'], exclude=[])
print(len(datasets))

22


# Delta F / F stack

In [6]:
import numpy as np
import multiprocessing as mp
from scipy.signal import detrend

def process_pixel_series(pixel_series):
    bp = np.linspace(0, len(pixel_series), 3, endpoint=True).astype('int')
    processed_series = detrend(gaussian_filter1d(pixel_series, 2), bp=bp)
    return processed_series

def worker(pixel_series_chunk):
    return [process_pixel_series(series) for series in pixel_series_chunk]

def parallel_detrend_stack(stack_data, N_processes=None):
    T, X, Y = stack_data.shape
    reshaped_stack = stack_data.reshape(T, X*Y)
    if N_processes is None:
        N_processes = mp.cpu_count()
    pixel_chunks = np.array_split(reshaped_stack.T, N_processes)
    pool = mp.Pool(processes=N_processes)
    processed_chunks = pool.map(worker, pixel_chunks)
    pool.close()
    pool.join()
    processed_stack = np.concatenate(processed_chunks).T.reshape(T, X, Y)
    return processed_stack

In [ ]:
for folder in datasets:
    if not any(identify_files(folder, ['dff_corrected_', '.tif'])):
        files = identify_files(folder, ['corrected', '.tif'])
        for file in tqdm(files, file=sys.stdout):
            stack = load_stack(folder + file).astype('float32')
            stack = parallel_dff_stack(stack, N_processes=30)
            stack[stack < 0] = 0
            stack -= np.min(stack)
            stack /= np.max(stack)
            stack *= 65535
            save_stack(folder + 'dff_' + file, stack)

 95%|████████████████████████████████████████▉  | 20/21 [13:22<00:40, 40.63s/it]

# Projections

In [7]:
from calimba.analysis.data import *

In [12]:
def get_multiplane_projections(directory, start=0, N_frames=None, batch_size=25, scale=(1, 1, 1), scale_side=5,
                               filter_noise=False, keywords=None, plane_marker='#'):
    b = batch_size
    if N_frames is None:
        if keywords is not None:
            files = identify_files(directory, keywords + ['.tif'])
        else:
            files = identify_files(directory, ['.tif'])
        image = Image.open(directory + files[0])
        N_frames = image.n_frames
    N_b = int(N_frames / b)

    frames_top = []

    for i in tqdm(range(N_b), file=sys.stdout):
        frame_ids = np.linspace(start + i * b, start + (i * b) + b - 1, b).astype('int')
        volumes = load_volumes(directory, frame_ids, keywords=keywords, plane_marker=plane_marker)
        frames_top.append(np.max(volumes, axis=1))
        
    frames_top = np.concatenate(frames_top, axis=0)

    frames_top = frames_top.astype('double')
    frames_top -= np.amin(frames_top)
    frames_top /= np.amax(frames_top)
    frames_top *= 255

    return frames_top.astype('uint8')

In [13]:
folder = datasets[7]

In [14]:
proj1 = get_multiplane_projections(folder, keywords=['dff_corrected', '.tif'])

100%|███████████████████████████████████████████| 35/35 [02:25<00:00,  4.17s/it]


In [15]:
save_stack('projection_top.tif', proj1)

# Generating brain activity gif

In [249]:
from PIL import Image

In [253]:
def figure_to_array(fig):
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    array = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    array.shape = (h, w, 4)
    array = np.roll(array, 3, axis=2)
    return array


def save_figures_as_gif(path, figures, fps=30):
    images = []
    for figure in figures:
        array = figure_to_array(figure)
        image = Image.fromarray(array)
        images.append(image)
    images[0].save(path, save_all=True, append_images=images[1:], optimize=False, duration=1000/fps, loop=0)

In [263]:
for j in range(len(rseries)):

    if j == 15:
    
        timeseries = rseries[j]
        
        figures = []
        for i in tqdm(range(timeseries.shape[1]), file=sys.stdout):
            
            state = timeseries[:, i]
            CAP = state.reshape(R, 1) @ state.reshape(1, R)
            
            fig, ax = plt.subplots(figsize=(3, 3))
            plot_matrix_communities(ax, CAP, communities, vmin=-5, vmax=5)
            ax.set_title(r"$t={}$ s".format(i))
            plt.close()
        
            figures.append(fig)
        
        save_figures_as_gif('Gifs/dynamic_FC_fish{}.gif'.format(j + 1), figures, fps=50)

100%|█████████████████████████████████████████| 892/892 [00:16<00:00, 53.52it/s]


/tmp/ipykernel_27729/4288959989.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  array = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
